In [1]:
import malaya_speech
import torch
from malaya_speech.utils import torch_featurization
import numpy as np
import json
from datasets import Audio

2023-09-28 05:26:18.598368: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 05:26:19.482141: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
`openai-whisper` is not available, native whisper processor is not available, will use huggingface processor instead.


[2023-09-28 05:26:29,094] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
# /home/ubuntu/bengali/data/LATEST_DATA_WAV2VEC2_DURATION.parquet
# 1. load the dataset
df = pd.read_parquet("/home/ubuntu/bengali/data/LATEST_DATA_WAV2VEC2_DURATION.parquet")
train, val = train_test_split(df, test_size=1, random_state=42)

In [2]:
class BengaliDataset(torch.utils.data.Dataset):

    SR = 16000

    def __init__(self, df):
        self.df = df
        self.audio = Audio(sampling_rate=self.SR)

    def __getitem__(self, idx):
        x = self.df.loc[idx]['path']
        y = self.df.loc[idx]['sentence']

        r = self.audio.decode_example(self.audio.encode_example(x))
        return r['array']

    def __len__(self):
        return len(self.df)

In [ ]:
train_dataset = BengaliDataset(val.reset_index(drop=True))

In [ ]:
dataloader = torch.utils.data.DataLoader(train_dataset, num_workers=4)

In [ ]:
train_dataset[0].dtype

In [ ]:
torch.Tensor(train_dataset[0]).dtype

In [ ]:
torch.__version__

In [ ]:
import torchaudio

In [ ]:
torchaudio.__version__

In [ ]:
torch.__version__

import torchaudio

torchaudio.__version__

from tqdm import tqdm

def generate_statistics(samples):
    E_x = 0
    E_x_2 = 0
    N = 0

    for sample in tqdm(samples):
#         print(sample[0].squeeze().dtype)
        mel_spec = torch_featurization.melspectrogram(sample[0].squeeze())
        scaled_mel_spec = torch_featurization.piecewise_linear_log(mel_spec)
        sum = scaled_mel_spec.sum(0)
        sq_sum = scaled_mel_spec.pow(2).sum(0)
        M = scaled_mel_spec.size(0)

        E_x = E_x * (N / (N + M)) + sum / (N + M)
        E_x_2 = E_x_2 * (N / (N + M)) + sq_sum / (N + M)
        N += M

    return E_x, (E_x_2 - E_x**2) ** 0.5

mean, stddev = generate_statistics(iter(dataloader))

json_str = json.dumps({"mean": mean.tolist(), "invstddev": (1 / stddev).tolist()}, indent=2)
with open('malay-stats.json', "w") as f:
    f.write(json_str)

In [ ]:
mean, stddev = generate_statistics(iter(dataloader))

In [ ]:
json_str = json.dumps({"mean": mean.tolist(), "invstddev": (1 / stddev).tolist()}, indent=2)
with open('malay-stats.json', "w") as f:
    f.write(json_str)